# Cleaning ___npidata___ (Taxonomy Switch Problem)

## 1. Importing Stuff

In [118]:
# importing stuff
import pandas as pd
import sqlite3
from tqdm.notebook import tqdm

# showing lots of rows and columns
pd.set_option('display.max_columns', 999)
pd.set_option('display.max_rows', 999)

# hiding annoying pink warning boxes
import warnings
warnings.filterwarnings("ignore")

# Trying with Chunking

In [ ]:
#appending all into df
chunks = pd.read_csv('data/NPPES_Data_Dissemination_February_2021/npidata_pfile_20050523-20210207.csv', chunksize = 100000)

npidata = pd.DataFrame([])

for chunk in chunks:
    npidata = npidata.append(chunk[chunk['Provider Business Practice Location Address State Name'] == 'TN'])
    
npidata.head()

## 2. Reading in as many rows as possible

In [168]:
#testing how much I can load in
npidata = pd.read_csv('data/NPPES_Data_Dissemination_February_2021/npidata_pfile_20050523-20210207.csv', 
                          nrows = 300000)

In [169]:
npidata = npidata[npidata['Provider Business Practice Location Address State Name'] == 'TN']

In [170]:
npidata['Provider Business Practice Location Address Postal Code'] = npidata['Provider Business Practice Location Address Postal Code'].astype(int)

In [171]:
npidata.head()

,NPI,Entity Type Code,Replacement NPI,Employer Identification Number (EIN),Provider Organization Name (Legal Business Name),Provider Last Name (Legal Name),Provider First Name,Provider Middle Name,Provider Name Prefix Text,Provider Name Suffix Text,Provider Credential Text,Provider Other Organization Name,Provider Other Organization Name Type Code,Provider Other Last Name,Provider Other First Name,Provider Other Middle Name,Provider Other Name Prefix Text,Provider Other Name Suffix Text,Provider Other Credential Text,Provider Other Last Name Type Code,Provider First Line Business Mailing Address,Provider Second Line Business Mailing Address,Provider Business Mailing Address City Name,Provider Business Mailing Address State Name,Provider Business Mailing Address Postal Code,Provider Business Mailing Address Country Code (If outside U.S.),Provider Business Mailing Address Telephone Number,Provider Business Mailing Address Fax Number,Provider First Line Business Practice Location Address,Provider Second Line Business Practice Location Address,Provider Business Practice Location Address City Name,Provider Business Practice Location Address State Name,Provider Business Practice Location Address Postal Code,Provider Business Practice Location Address Country Code (If outside U.S.),Provider Business Practice Location Address Telephone Number,Provider Business Practice Location Address Fax Number,Provider Enumeration Date,Last Update Date,NPI Deactivation Reason Code,NPI Deactivation Date,NPI Reactivation Date,Provider Gender Code,Authorized Official Last Name,Authorized Official First Name,Authorized Official Middle Name,Authorized Official Title or Position,Authorized Official Telephone Number,Healthcare Provider Taxonomy Code_1,Provider License Number_1,Provider License Number State Code_1,Healthcare Provider Primary Taxonomy Switch_1,Healthcare Provider Taxonomy Code_2,Provider License Number_2,Provider License Number State Code_2,Healthcare Provider Primary Taxonomy Switch_2,Healthcare Provider Taxonomy Code_3,Provider License Number_3,Provider License Number State Code_3,Healthcare Provider Primary Taxonomy Switch_3,Healthcare Provider Taxonomy Code_4,Provider License Number_4,Provider License Number State Code_4,Healthcare Provider Primary Taxonomy Switch_4,Healthcare Provider Taxonomy Code_5,Provider License Number_5,Provider License Number State Code_5,Healthcare Provider Primary Taxonomy Switch_5,Healthcare Provider Taxonomy Code_6,Provider License Number_6,Provider License Number State Code_6,Healthcare Provider Primary Taxonomy Switch_6,Healthcare Provider Taxonomy Code_7,Provider License Number_7,Provider License Number State Code_7,Healthcare Provider Primary Taxonomy Switch_7,Healthcare Provider Taxonomy Code_8,Provider License Number_8,Provider License Number State Code_8,Healthcare Provider Primary Taxonomy Switch_8,Healthcare Provider Taxonomy Code_9,Provider License Number_9,Provider License Number State Code_9,Healthcare Provider Primary Taxonomy Switch_9,Healthcare Provider Taxonomy Code_10,Provider License Number_10,Provider License Number State Code_10,Healthcare Provider Primary Taxonomy Switch_10,Healthcare Provider Taxonomy Code_11,Provider License Number_11,Provider License Number State Code_11,Healthcare Provider Primary Taxonomy Switch_11,Healthcare Provider Taxonomy Code_12,Provider License Number_12,Provider License Number State Code_12,Healthcare Provider Primary Taxonomy Switch_12,Healthcare Provider Taxonomy Code_13,Provider License Number_13,Provider License Number State Code_13,Healthcare Provider Primary Taxonomy Switch_13,Healthcare Provider Taxonomy Code_14,Provider License Number_14,Provider License Number State Code_14,Healthcare Provider Primary Taxonomy Switch_14,Healthcare Provider Taxonomy Code_15,Provider License Number_15,Provider License Number State Code_15,Healthcare Provider Primary Taxonomy Switch_15,Other Provider Identifier_1,Other Provider Identifier Type Code_1,Other Provider Ide

In [172]:
npidata.shape

(7127, 330)

# Trying what Michael Said...

In [173]:
def Get_Taxonomy_Code(df_row):
    j = 1
    switch_code = 0
    while j <= 15:
        column_name = "Healthcare Provider Primary Taxonomy Switch_" + str(j)
        if df_row[column_name] == 'Y':
            switch_code = j
            j = 16
        j+=1
    if switch_code != 0:
        col_1 = "Healthcare Provider Taxonomy Code_" + str(switch_code)
        return df_row[col_1]
    else:
        return -1


#appending all into df
chunks = pd.read_csv('data/NPPES_Data_Dissemination_February_2021/npidata_pfile_20050523-20210207.csv', chunksize = 100000)

npidata = pd.DataFrame([])

for chunk in chunks:
    npidata = npidata.append(chunk[chunk['Provider Business Practice Location Address State Name'] == 'TN'])
    
npidata.head()

In [175]:
npidata['Healthcare_Provider_Taxonomy_Code'] = npidata.apply(Get_Taxonomy_Code, axis = 1)

In [176]:
#getting rid of all that have no Switch Code = 'Y'
npidata = npidata[npidata['Healthcare_Provider_Taxonomy_Code'] != -1 ]

In [177]:
#Subsetting for needed columns
#choosing not to include middle name, Replacement NPI, and (EIN) b/c they are mostly NaN (Can add later if needed)
npidata = npidata[['NPI','Entity Type Code','Provider Organization Name (Legal Business Name)',
                   'Provider Last Name (Legal Name)','Provider First Name',
                   'Provider First Line Business Practice Location Address',
                   'Provider Second Line Business Practice Location Address',
                   'Provider Business Practice Location Address City Name',
                   'Provider Business Practice Location Address Postal Code',
                   'Provider Credential Text','Provider Gender Code',
                  'Healthcare_Provider_Taxonomy_Code']]

In [178]:
#Renaming Columns
npidata = npidata.rename(columns = {"Entity Type Code": "Entity_Type_Code",
                                    "Provider Organization Name (Legal Business Name)":"Provider_Organization_Name",
                                   "Provider Last Name (Legal Name)":"Provider_Last_Name",
                                   "Provider First Name":"Provider_First_Name",
                                   "Provider Credential Text":"Provider_Credential_Text",
                                   "Provider Gender Code":"Gender",
                                   'Provider First Line Business Practice Location Address':'First_Line_Address',
                                   'Provider Second Line Business Practice Location Address':'Second_Line_Address',
                                   'Provider Business Practice Location Address City Name':'Address_City',
                                   'Provider Business Practice Location Address Postal Code':'Address_Postal_Codes'})

In [179]:
npidata.head()

,NPI,Entity_Type_Code,Provider_Organization_Name,Provider_Last_Name,Provider_First_Name,First_Line_Address,Second_Line_Address,Address_City,Adress_Postal_Codes,Provider_Credential_Text,Gender,Healthcare_Provider_Taxonomy_Code
17,1841293891,1.0,NaN,GIBBS,ELMER,49 CLEVELAND ST 310,NaN,CROSSVILLE,385552854,M.D.,M,208600000X
28,1659374601,1.0,NaN,OBERDICK,WENDY,105 W STONE DR,STE 1F,KINGSPORT,376603365,MD,F,207Q00000X
48,1134122187,1.0,NaN,RUDNICKE,CHERYL,250 25TH AVE N,STE 412,NASHVILLE,372031632,CRNP,F,363L00000X
67,1003819046,1.0,NaN,NYLANDER,BARBARA,345 23RD AVE N,SUITE 209,NASHVILLE,372031513,M.D.,F,207VG0400X
76,1750384780,1.0,NaN,PERRIGIN,JULIE,219 CHURCH ST,NaN,DICKSON,370551303,MD,F,207Q00000X


## 3. Making a new column that gives the number where the Taxonomy_Switch = 'Y'

In [66]:
npidata['taxonomy_switch_num'] = 0

#looping through each row in df
for i in range(npidata.shape[0]):
    j = 1
    while j <= 15:
        column_name = "Healthcare Provider Primary Taxonomy Switch_" + str(j)
        if npidata[column_name].iloc[i] == 'Y':
            npidata['taxonomy_switch_num'].iloc[i] = j
            j = 16
        j+=1

## 4. Getting rid of all rows that did not have a Taxonmy_Switch = 'Y' anywhere

In [67]:
npidata = npidata[npidata['taxonomy_switch_num'] != 0]

## 5. Creating 3 new columns with the provider info to keep

In [68]:
# Setting up New Columns to Eventually Keep
npidata['Healthcare_Provider_Taxonomy_Code'] = 'Nothing Yet'
npidata['Provider_License_Number'] = 'Nothing Yet'
npidata['Provider_License_Number_State'] = 'Nothing Yet'

## 6. Filling in new columns with the correct info based on the taxonomy_switch number

In [69]:
# iterating through the number of rows
for k in range(npidata.shape[0]):
    col_1 = "Healthcare Provider Taxonomy Code_" + str(npidata['taxonomy_switch_num'].iloc[k])
    npidata['Healthcare_Provider_Taxonomy_Code'].iloc[k] = npidata[col_1].iloc[k]
    
    #col_2 = "Provider License Number_" + str(npidata['taxonomy_switch_num'].iloc[k])
    #npidata['Provider_License_Number'].iloc[k] = npidata[col_2].iloc[k]
    
    #col_3 = "Provider License Number State Code_" + str(npidata['taxonomy_switch_num'].iloc[k])
    #npidata['Provider_License_Number_State'].iloc[k] = npidata[col_3].iloc[k]

## 7. Subsetting the df for only the needed columns

In [70]:
#Subsetting for needed columns
#choosing not to include middle name, Replacement NPI, and (EIN) b/c they are mostly NaN (Can add later if needed)
npidata = npidata[['NPI','Entity Type Code','Provider Organization Name (Legal Business Name)',
                   'Provider Last Name (Legal Name)','Provider First Name',
                   'Provider First Line Business Practice Location Address',
                   'Provider Second Line Business Practice Location Address',
                   'Provider Business Practice Location Address City Name',
                   'Provider Business Practice Location Address Postal Code',
                   'Provider Credential Text','Provider Gender Code','taxonomy_switch_num',
                  'Healthcare_Provider_Taxonomy_Code','Provider_License_Number','Provider_License_Number_State']]

## 8. Renaming the columns so the df is easier to work with

In [71]:
#Renaming Columns
npidata = npidata.rename(columns = {"Entity Type Code": "Entity_Type_Code",
                                    "Provider Organization Name (Legal Business Name)":"Provider_Organization_Name",
                                   "Provider Last Name (Legal Name)":"Provider_Last_Name",
                                   "Provider First Name":"Provider_First_Name",
                                   "Provider Credential Text":"Provider_Credential_Text",
                                   "Provider Gender Code":"Gender",
                                   'Provider First Line Business Practice Location Address':'First_Line_Address',
                                   'Provider Second Line Business Practice Location Address':'Second_Line_Address',
                                   'Provider Business Practice Location Address City Name':'Address_City',
                                   'Provider Business Practice Location Address Postal Code':'Adress_Postal_Codes'})

## 9. Taking a look at the clean data

In [72]:
#Looking at clean data
npidata.tail(10)

,NPI,Entity_Type_Code,Provider_Organization_Name,Provider_Last_Name,Provider_First_Name,First_Line_Address,Second_Line_Address,Address_City,Adress_Postal_Codes,Provider_Credential_Text,Gender,taxonomy_switch_num,Healthcare_Provider_Taxonomy_Code,Provider_License_Number,Provider_License_Number_State
199551,1235110206,1.0,NaN,STONE,BETTY,309 N BROAD ST,NaN,NEW TAZEWELL,378256600,P.A.C.,F,1,363A00000X,PA0000000671,TN
199577,1396726360,1.0,NaN,DUGGER,GREGORY,403 E COLLEGE ST,NaN,PULASKI,384784315,NaN,M,1,1223G0001X,DS2544,TN
199617,1811978737,1.0,NaN,CARDEN,KELLY,300 20TH AVE N STE G8,NaN,NASHVILLE,372032225,MD,F,4,207RS0012X,MD0000042892,TN
199681,1972584993,1.0,NaN,MILAM,JACQUELINE,490 DUNLOP LN,NaN,CLARKSVILLE,370405007,MD,F,1,207R00000X,21903,TN
199685,1417938432,1.0,NaN,ENDSLEY,JOHN,311 LANDRUM PLACE,STE 100,CLARKSVILLE,37043,MD,M,1,207RN0300X,23560,TN
199688,1144201161,1.0,NaN,CARRIGAN,VERNON,490 DUNLOP LN,NaN,CLARKSVILLE,370405007,MD,M,1,207R00000X,10268,TN
199689,1053392076,1.0,NaN,VETTER,MICHAEL,332 6TH ST,NaN,BRISTOL,376202201,D.M.D.,M,1,1223G0001X,7941,TN
199758,1932180841,1.0,NaN,EAGLE,MICHAEL,206 BEDFORD WAY,NaN,FRANKLIN,370645526,P.A.-C,M,1,363AS0400X,PA1419,TN
199827,1912988841,1.0,NaN,POLETAJEV,VICTOR,5107 BRYSON TRL,NaN,MURFREESBORO,371293063,D.C.,M,4,111N00000X,DC-001053,TN
199968,1013998038,1.0,NaN,LAMB,MELISSA,229 HEDRICK DR,NaN,NEWPORT,378212902,"APRN,BC",F,1,363LF0000X,11579,TN


- Need to keep address and Classification (Classification might be in different df)
- Possible narrow down to 150 zipcodes in nashville

## 10. Exporting Data to CSV so we have something to start working with

In [73]:
npidata.to_csv(r'data/NPPES_Data_Dissemination_February_2021/npidata_clean.csv', header = True)